In [1]:
import pandas as pd
import os, sys
import csv
from pathlib2 import Path

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [4]:
def get_forum_files(forum_path, suffix):
    
    forum_path = Path(forum_path)
    all_files = [str(file) for file in forum_path.rglob('*.' + suffix)]
    
    return all_files

In [30]:
def get_forum_id(forum_path):
    
    forum_id = forum_path.split('/')[-1].split('_')[1]

    return forum_id

# forum_path = '/l/nx/data/deepspace/prisontalk/data/fid_583_posts.csv'
# fid = get_forum_id(forum_path)
# fid

In [106]:
def get_forum_ids(forum_paths):
    
    forum_ids = [int(get_forum_id(forum_path)) for forum_path in forum_paths]
    
    return forum_ids


# data_path = '/l/nx/data/deepspace/prisontalk/data/'
# forum_posts_path = 'forums/parsed/posts/'
# forum_files = get_forum_files(data_path + forum_posts_path, 'csv')
# file_fids = get_forum_ids(forum_files)
# print len(file_fids)

### RECORDS OF SCRAPED FORUMS DIFFER
The scraped forums were placed in a list called forum_ids.csv.  This list differs from the forum ids from the file names (e.g., fid_3807.csv). First, I checked to find out which forums differered from each.  The forums that were in forum_ids.csv but *not* in the file list were checked in a browser and pointed to forums.  The forum ids that were in the files but *not* in forum_ids.csv did not go to a forum.  I loaded the first two files on the list (fid_3807.csv and fid_3808.csv) into a datafame and discovered that fid's were the same. I realized that most likely due to scraper error, these posts were given unique file ids but were from the same forum.  While it would appear that I was able to restart the scraper where it left off, they all need to be checked.  As you'll see below, this only applies to three forums: 38, 44, and 110.  The remaining two forums (957 and 1627) are discussed below.

**forum_ids:** [38, 44, 110, 957, 1627]  
**file_ids:** [3807, 3808, 4405, 4406, 11005, 11007]

**38 General Prison Talk:** 3807, 3808  
**44 Husbands & Boyfriends in Prison:** 4405, 4406  
**110 Wives & Girlfriends in Prisonforum:** 11005, 11007

Forum 957 and 1627 required a login to scrape.  While the infrastructure to log in to scrape these forums is set up, a decision was made to skip these forums. Although they are public in the sense that anyone can sign up to read them, the required login may also be a sign that these forums are more sensiteive than others, and therefore could fall under 'private'.  The domestic violence forum (957) is sensitive enough to require it's own treatment.  It is not soley from the title what the sensitive nature of the life after prison forum is, but again, this could be a paper all on it's own.

**957:** Domestic Violence Forum Member Introductions & Stories _(Login Required)_  
**1627:** LASO: Life After Prison _(Login Required)_

In [ ]:
data_path = '/l/nx/data/deepspace/prisontalk/data/'
forum_posts_path = 'forums/parsed/posts/'

forum_list = pd.read_csv(data_path + 'forums/forum_list.csv')
forum_files = get_forum_files(data_path + forum_posts_path, 'csv')

In [107]:
file_ids = get_forum_ids(forum_files) # ID's FROM PARSING THE FILES
df_fids = list(forum_ids['fid']) # ID'S FROM THE forum_list DATAFRAME

diff_df = [fid for fid in df_fids if fid not in file_fids] # FIDS IN DF THAT ARE NOT IN THE FILES
diff_file = [fid for fid in file_fids if fid not in df_fids] # FIDS IN THE FILES THAT ARE NOT IN THE DF

diff_df.sort()
diff_file.sort()

print len(diff_df) # 5
print diff_df # [38, 44, 110, 957, 1627]
print ''
print len(diff_file) # 6
print diff_file # [3807, 3808, 4405, 4406, 11005, 11007]

In [130]:
# NOT VERY ELEGANT, TRYING TO MOVE ON WITH MY LIFE
forum_parts = [['3807', '3808'], ['4405', '4406'], ['11005', '11007']]
fids = ['38', '44', '110']

for forum_part, fid in zip(forum_parts, fids):
    
    print 'COMBINING FORUM', fid + ':', forum_part[0], '&', forum_part[1]
    
    file_one = 'fid_' + forum_part[0] + '_posts.csv'
    file_two = 'fid_' + forum_part[1] + '_posts.csv'
    full_file = 'fid_' + fid + '_posts.csv'
    
    part_one = pd.read_csv(data_path + forum_posts_path + file_one)
    part_two = pd.read_csv(data_path + forum_posts_path + file_two)
    
    full_forum = pd.concat([part_one, part_two]) # CONCAT TO FULL FORUM
    
    rows_one = part_one.shape[0]
    rows_two = part_two.shape[0]
    total_rows = rows_one + rows_two
    
    if total_rows == full_forum.shape[0]:
        
        full_forum.to_csv(data_path + forum_posts_path + full_file, index=False)
    
    print '\nPart 1:', rows_one, '\nPart 2:', rows_two, '\nBoth:', total_rows, '\nFull:', full_forum.shape[0], '\n'

COMBINING FORUM 38: 3807 & 3808

Part 1: 72817 
Part 2: 163249 
Both: 236066 
Full: 236066 

COMBINING FORUM 44: 4405 & 4406

Part 1: 228870 
Part 2: 481595 
Both: 710465 
Full: 710465 

COMBINING FORUM 110: 11005 & 11007

Part 1: 16472 
Part 2: 16476 
Both: 32948 
Full: 32948 



In [97]:
# IMPORT CSV, SET 'DATE' COLUMN TO THE INDEX AS DATETIME
fid_3807 = pd.read_csv(data_path + forum_posts_path + 'fid_3807_posts.csv', parse_dates=['date'], infer_datetime_format=True, index_col='date')
fid_3808 = pd.read_csv(data_path + forum_posts_path + 'fid_3808_posts.csv', parse_dates=['date'], infer_datetime_format=True, index_col='date')

fid_38 = pd.concat([fid_3807, fid_3808]) # CONCAT TO FULL FORUM

In [62]:
forum_ids.sort_values('fid')

,fid,name,threads,posts,description,retrieval
1,38,Introduce Yourself!,25870,232069,Please stop by and introduce yourself and shar...,C
0,39,General Prison Talk,30154,468915,Does your issue not fit into another forum? Po...,C
18,44,Husbands & Boyfriends in Prison,31816,698529,"For everyone who has a husband, boyfriend or m...",C
40,51,Straight Talk,639,8682,The general Ex-Offender discussion forum. If ...,C
31,52,"Probation, Parole & Supervision",6009,33603,"Having problems (or need info) with Probation,...",C
36,53,Employment w/a Criminal Record,1395,8467,Finding post-incarceration employment can be a...,C
16,65,Juvenile,452,2833,Discussion of everything related to minors in ...,C
27,66,Death Row & Capital Punishment Discussions,4862,37391,Discussions relating to inmates on Death Row /...,C
3,72,Prison & Criminal Legal Help!,10295,62181,"Ask questions, get opinions, and find resource...",C
13,75,Parents with Children in Prison,8714,192088,For the parents of prisoners,C


In [131]:
forum_ids.shape[0]

43

In [6]:
data_path = '/l/nx/data/deepspace/prisontalk/data/'
forum_posts_path = 'forums/parsed/posts/'



In [38]:
forum_ids = pd.read_csv(data_path + 'forums/forum_ids.csv')

forum_ids.head()

,fid,name,threads,posts,description,retrieval
0,39,General Prison Talk,30154,468915,Does your issue not fit into another forum? Po...,C
1,38,Introduce Yourself!,25870,232069,Please stop by and introduce yourself and shar...,C
2,91,Headed to Prison,1989,15799,Dedicated to those who are facing incarceratio...,C
3,72,Prison & Criminal Legal Help!,10295,62181,"Ask questions, get opinions, and find resource...",C
4,202,Drug & Alcohol Treatment & Rehabilitation,1732,17876,For those whose lives have been touched by add...,C


In [63]:
forum_list = pd.read_csv(data_path + 'forums/forum_list.csv')

forum_list.sort_values('number')

,number,name,threads,posts,description
0,33,"PTO News, Information & Events",321,"6,673","What's new, and what is coming to our web comm..."
1,38,Introduce Yourself!,"25,870","232,069",Please stop by and introduce yourself and shar...
21,39,General Prison Talk,"30,154","468,915",Does your issue not fit into another forum? Po...
22,44,Husbands & Boyfriends in Prison,"31,816","698,529","For everyone who has a husband, boyfriend or m..."
18,47,General Health Care,"1,465","11,983","Prison Health Care.. Medical, medications an..."
15,50,The War on Drugs - and the results of it,767,"7,343","A war against drugs, or against families?"
49,51,Straight Talk,639,"8,682",The general Ex-Offender discussion forum. If ...
43,52,"Probation, Parole & Supervision","6,009","33,603","Having problems (or need info) with Probation,..."
44,53,Employment w/a Criminal Record,"1,395","8,467",Finding post-incarceration employment can be a...
96,56,TEXAS,"42,898","455,501",Topics and conversations dealing with the Stat...


In [65]:
forum_list[forum_list['number'].isin(['3807', '3808', '4405', '4406', '11005', '11007'])]

,number,name,threads,posts,description


In [ ]:
#### RETRIEVE INDIVIDUAL FILES

In [27]:
forum_files = get_forum_files(data_path + forum_posts_path, 'csv')
print len(forum_files)

44


In [21]:
forum_dfs = []

for forum_file in forum_files:
    
    forum_df = pd.read_csv(forum_file, parse_dates=['date'], infer_datetime_format=True, index_col='date')
    forum_dfs.append(forum_df)

forums = pd.concat(forum_dfs)

In [ ]:
locations = pivot[loc_cols]